In [348]:
import evennia
evennia._init()

### Mob Difficulty Calculations for HP, MP, SP

main code used to experiment as well as act as a guideline to calculate
all vital attributes

In [404]:
import numpy as np
import math

import matplotlib.pyplot as plt
from evennia.contrib.dice import roll_dice

class ndict(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        for k,v in self.items():
            setattr(self, k, v)
    def __setattr__(self, k, v):
        super().__setattr__(k,v)
        self[k] = v
    
    def __getattr__(self, k):
        return self[k]

def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)
    
class MobDifficulty(IntEnum):
    wimpy = 0
    easy = 1
    medium = 2
    hard = 3
    insane = 4
    chaotic = 5

    def members(return_dict=False):
        if return_dict:
            return {
                k.lower(): v
                for k, v, in MobDifficulty._member_map_.items()
            }
        return list(reversed(MobDifficulty._member_map_.keys()))
    
    def next(self):
        value = self.value + 1
        if value > MobDifficulty.chaotic.value:
            return self
        
        return MobDifficulty(value)
    
class AutoMobScaling:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)
    def translate(self, value, leftMin, leftMax, rightMin, rightMax):
        # Figure out how 'wide' each range is
        leftSpan = leftMax - leftMin
        rightSpan = rightMax - rightMin

        # Convert the left range into a 0-1 range (float)
        valueScaled = float(value - leftMin) / float(leftSpan)

        # Convert the 0-1 range into a value in the right range.
        return rightMin + (valueScaled * rightSpan)
    @property
    def base(self):
        return (self.base_mult * np.log2(self.olvl)) * self.olvl
    @property
    def diff_mod(self):
        return self.diff_mult ** (self.diff.value-1)
    
    def __repr__(self):
        return str(self.calc())

    def calc_hp(self):
        val = int((self.base * np.log(self.end)) * self.diff_mod)
        return max(val, 3)

    def calc_mp(self):
        val = int((self.base * np.log(
            (self.int + self.wp) / 2)) * self.diff_mod)
        return max(val, 3)

    def calc_sp(self):
        val = int((self.base * np.log(
            (self.end + self.agi) / 2)) * self.diff_mod)
        return max(val, 3)
    
    def calc_dam(self):
        """returns tuple of dice roller
        num, size, mod, (avg)
        ex: [num]D[size] + mod
        """
        old_min = MobDifficulty.wimpy.value
        old_max = MobDifficulty.chaotic.value
        avg = self.calc_hp() / self.translate(
            self.diff.value, old_min, old_max, old_max + 1, old_min + 1)
        
        num = max(1, int(avg * 1.1))
        vals = []
        threshold = 3
        while not vals:
            for x in range(1, 200):  # x
                for y in range(1, 200):  # y
                    tavg = int((((y + 1) / 2) * x))
                    if abs(tavg - num) < threshold:
                        vals.append((x, y))
            threshold += 10
        v = None
        mwp = len(vals) // 2
        if vals:
            v = vals[mwp]
        if v:  
            dam_num = v[0]
            dam_size = v[1]
            dam_max = int(dam_num * dam_size)
            return dam_max, int(avg_roll(dam_num, dam_size)), (round(avg_roll(dam_num, dam_size)/self.calc_hp(), 2))
        return 0,0

    def calc_hit(self):
        hit_roll = min(
            99,
            int(self.translate(self.olvl, 1, 250, 40, 100)) +
            int(self.translate(self.diff.value, 0, 5, -5, 25)))
        return hit_roll

    
stat_names = ['str', 'end','agi', 'int', 'wp','prc', 'prs', 'lck']
diff_mod = 2
diffs = ('wimpy', 'easy', 'medium', 'hard','insane')

m = ndict(
olvl = 1,
base_mult = 1.5,
base_stat = 6,
num_dice = 2,
size_dice = 5,
diff = MobDifficulty.wimpy,
diff_mult=2,
)

LEVELS = 250
BATCH = 25
SAMPLES = (LEVELS, BATCH)
LEVEL_START = 1

dice = np.vectorize(lambda x: x + roll_dice(m.num_dice, m.size_dice))
stats = dice(np.full((1,8), fill_value=m.base_stat, dtype=np.float64))
stats = np.tile(stats, (SAMPLES[0],1))

for _idx, diff in enumerate(diffs):
    base = stats

    results = []
    print(f"{m.diff.name.capitalize()} {m.diff_mult ** m.diff.value}x")
    
    for level, data in enumerate(base):
        if (level+LEVEL_START) % SAMPLES[1] == 0 or (level) == 0:
            n = ndict({**m, **ndict(zip(stat_names, data))})
            n.olvl = level+LEVEL_START
            auto = AutoMobScaling(**n)
            results.append([n.olvl, auto.calc_hp(), auto.calc_mp(), auto.calc_sp(), auto.calc_dam()]+data.tolist())
            
    m.diff=m.diff.next()

    from tabulate import tabulate
    print(tabulate(results,headers=['hp','wp', 'sp', 'dam']+stat_names)+'\n')
#     break


Wimpy 1x
       hp    wp    sp  dam                  str    end    agi    int    wp    prc    prs    lck
---  ----  ----  ----  -----------------  -----  -----  -----  -----  ----  -----  -----  -----
  1     3     3     3  (5, 3, 1.0)           11     13     15     13    15     13     13     11
 25   223   229   229  (72, 39, 0.17)        11     13     15     13    15     13     13     11
 50   542   558   558  (190, 100, 0.18)      11     13     15     13    15     13     13     11
 75   898   924   924  (306, 162, 0.18)      11     13     15     13    15     13     13     11
100  1278  1315  1315  (442, 234, 0.18)      11     13     15     13    15     13     13     11
125  1675  1723  1723  (588, 308, 0.18)      11     13     15     13    15     13     13     11
150  2085  2146  2146  (736, 384, 0.18)      11     13     15     13    15     13     13     11
175  2508  2580  2580  (884, 459, 0.18)      11     13     15     13    15     13     13     11
200  2940  3025  3025  (1044, 5

### Experimentation with calcing dam_roll statistics based on level

The base assumption.

If two equal level mobs are fighting with same hp, wp, sp and doing nothing special. Their should be a 50/50 chance the other wins in 5 rounds. From this baseline, we can calculate the potential dam_roll and hit based on level

Each tier of difficulty of the above situation falls in favor on the mob by a 15% difference.. 

Chance of Winning for Player

| Difficulty | Player |  Mob
|------------|--------|-----
|  wimpy     |  90    | 10
|  easy      |  50    | 50
|  medium    |  40    | 60
|  hard      |  20    | 80
|  insane    |  10    | 90
|  chaotic   |   5    | 95


In [350]:
def bench(samples, num, size, mod=0):
    x = [roll_dice(num, size, ('+', mod)) for _ in range(samples)]
    return np.array(x).mean()

def test(val, diff):
    avg = val/translate(diff, 0,5,6,1)
    num = avg/5
    size = 9.0
    print(num, size, avg)
    return int(bench(1000, num,size))

test(558, 1)


22.32 9.0 111.6


109

In [351]:


avg_roll(6,95,0)*2

576

In [399]:
def avg_roll(num, size, mod=0):
    #( ( Max Die Roll + 1 ) / 2 ) * Number of Same-Sided Dice
    return int((((size+1)/2)*num)+mod)

end_value = 10
# for value in range(0,end_value):
value = 8414
l = [f'v: {value}']
vals = []
for x in range(1,200): # x
    for y in range(1,200): # y
#             if y > x:
#                 break
        if abs(avg_roll(x,y,0)-value) < 5:
            print('here')
            l.append(f"  {x}d{y}")
            vals.append((x,y))
v = None
mwp = len(vals) // 2
if vals:
    v = vals[mwp]
# print(", ".join(l)+" " + (str(v)) )
    print(f"use dice roll: {v[0]}d{v[1]}")
    print(f"avg is: {avg_roll(v[0], v[1],0)}")


here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
use dice roll: 138d121
avg is: 8418


In [353]:
avg_roll(200,200,0)

20100